### 2016-01 Recuperatorio

Una red social almacena el contenido de los chats entre sus usuarios
en un RDD que tiene registros con el siguiente formato: (chat_id,
user_id, nickname, text). Queremos saber cuál es el usuario (user_id)
que mas preguntas hace en sus chats, contabilizamos una pregunta por
cada caracter “?” que aparezca en el campo text. Programar en Spark
un programa que identifique al usuario preguntón.

In [1]:
chats = [
    (1, 1, 'damu', 'Qué es esto?'),
    (2, 2, 'martin', 'Un chat!'),
    (3, 1, 'damu', 'Ahhh! Y de donde salio? Whatsapp?'),
    (4, 2, 'martin', 'Sí! Cómo sabias?'),
    (5, 1, 'damu', 'Adivine'),
    (6, 3, 'luis', 'Hola!')
]

In [2]:
data = sc.parallelize(chats);

In [3]:
data.collect()

[(1, 1, 'damu', 'Qu\xc3\xa9 es esto?'),
 (2, 2, 'martin', 'Un chat!'),
 (3, 1, 'damu', 'Ahhh! Y de donde salio? Whatsapp?'),
 (4, 2, 'martin', 'S\xc3\xad! C\xc3\xb3mo sabias?'),
 (5, 1, 'damu', 'Adivine'),
 (6, 3, 'luis', 'Hola!')]

In [10]:
data.map(lambda x: (x[1], x[3].count('?')))\
    .reduceByKey(lambda x, y: x+y)\
    .reduce(lambda x, y: x if (x[1] > y[1]) else y)

(1, 3)

### 2016-01 Parcial

UBER almacena en un cluster todos los datos sobre el movimiento y
viajes de todos sus vehículos. Existe un proceso que nos devuelve un
RDD llamado trip_summary con los siguientes campos: (driver_id,
car_id, trip_id, customer_id, date (YYYYMMDD), distance_traveled),
Programar usando Py Spark un programa que nos indique cual fue el
conductor con mayor promedio de distancia recorrida por viaje para
Abril de 2016.

In [11]:
trips = [
    (1, 1, 1, 1, '20160101', 10),
    (2, 2, 2, 2, '20160202', 20),
    (1, 1, 3, 1, '20160402', 15),
    (1, 1, 4, 3, '20160405', 20),
    (2, 2, 5, 4, '20160410', 25),
    (3, 3, 6, 3, '20160415', 15),
    (2, 2, 7, 1, '20160420', 40),
    (3, 3, 8, 2, '20160505', 80)
]

In [12]:
data = sc.parallelize(trips);

In [13]:
data.collect()

[(1, 1, 1, 1, '20160101', 10),
 (2, 2, 2, 2, '20160202', 20),
 (1, 1, 3, 1, '20160402', 15),
 (1, 1, 4, 3, '20160405', 20),
 (2, 2, 5, 4, '20160410', 25),
 (3, 3, 6, 3, '20160415', 15),
 (2, 2, 7, 1, '20160420', 40),
 (3, 3, 8, 2, '20160505', 80)]

In [26]:
data.filter(lambda x: (x[4] > '20160400') and (x[4] < '20160500'))\
    .map(lambda x: (x[0], (1, x[5])))\
    .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))\
    .map(lambda x: (x[0], x[1][1]/x[1][0]))\
    .reduce(lambda x, y: x if x[1] > y[1] else y)
    
# El último map no es necesario, agregado solo por claridad. 
# Sin el map lo único que cambia es la comparación que se hace en el reduce.

(2, 32)

### 2015-02 2do Recuperatorio

Un telescopio registra automaticamente la luminosidad de distintas
estrellas generando un RDD con registros de tipo (star_id,
magnitude,spectral_type, timestamp). Queremos obtener un listado de
estrellas que tienen el mismo tipo espectral e igual promedio de
magnitud una vez redondeado el mismo a un decimal. El resultado
debe ser una lista en donde cada elemento de la lista es una lista de ids
de estrellas similares.

In [27]:
stars = [
    (1, 5, 1, 1),
    (2, 10, 1, 1),
    (3, 6, 1, 1),
    (4, 5.5, 2, 1),
    (1, 6, 1, 2),
    (2, 9, 1, 2),
    (3, 5, 1, 2),
    (1, 5.5, 1, 3),
    (2, 11, 1, 3),
    (3, 5.5, 1, 3)
]

In [28]:
data = sc.parallelize(stars);

In [29]:
data.collect()

[(1, 5, 1, 1),
 (2, 10, 1, 1),
 (3, 6, 1, 1),
 (4, 5.5, 2, 1),
 (1, 6, 1, 2),
 (2, 9, 1, 2),
 (3, 5, 1, 2),
 (1, 5.5, 1, 3),
 (2, 11, 1, 3),
 (3, 5.5, 1, 3)]

In [37]:
data.map(lambda x: (x[0], (x[2], x[1], 1)))\
    .reduceByKey(lambda x, y: (x[0], x[1]+y[1], x[2]+y[2]))\
    .map(lambda x: ((x[1][0], x[1][1]/x[1][2]), x[0]))\
    .groupByKey()\
    .map(lambda x: (x[0], list(x[1])))\
    .collect()
    
# El último map es simplemente para convertir la sequencia de spark en una lista de python para poder verla

[((2, 5.5), [4]), ((1, 10), [2]), ((1, 5.5), [1, 3])]

### 2015-01 2do Recuperatorio

Tenemos una colección de documentos (textos) almacenados en un cluster. Se quiere establecer un ranking de los patrones mas frecuentes para la aparición de letras en las palabras. Por ejemplo “sister”, “ejects” , “ninety” y “amazon” responden al patrón “abacde”. Programar en map-reduce un programa que genere como resultado un
listado de tipo (patron, frecuencia) indicando cuántas veces aparece cada patrón en la colección de documentos.

In [ ]:
textFile = sc.textFile("README.md")
words = textFile.flatMap(lambda line: line.split())
words.collect()

In [64]:
def pattern(word):
    letters = 'abcdefghijklmnopqrstuvwxyz'
    pat = ''
    found = ''
    for letter in word:
        if letter in found:
            pat += pat[found.index(letter)]
        else:
            found += letter
            if len(found) > len(letters):
                pat += '?'
            else:
                pat += letters[len(found)-1]
    return pat
   
words.map(lambda x: (pattern(x), 1))\
     .reduceByKey(lambda x, y: x+y)\
     .takeOrdered(10, lambda x: -x[1])

[('abc', 85),
 ('ab', 58),
 ('abcd', 39),
 ('abcde', 37),
 ('abcdef', 16),
 ('abcdefg', 16),
 ('a', 12),
 ('aa', 8),
 ('abcdefgh', 8),
 ('abcdde', 6)]